In [1]:
import os
import subprocess

REPO_URL = "https://github.com/andrasjuice66/deloitte_rag_task.git"
repo_name = os.path.splitext(os.path.basename(REPO_URL))[0]
repo_dir = os.path.join(os.getcwd(), repo_name)

if not os.path.exists(repo_dir):
    subprocess.run(["git", "clone", REPO_URL, repo_dir], check=True)
else:
    subprocess.run(["git", "-C", repo_dir, "pull", "--ff-only"], check=True)

print(f"Repository ready at: {repo_dir}")


Cloning into '/Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task'...


Repository ready at: /Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task


In [2]:
# Ensure we operate inside the cloned repo so relative paths stay the same
import os, sys

REPO_URL = "https://github.com/andrasjuice66/deloitte_rag_task.git"
repo_name = os.path.splitext(os.path.basename(REPO_URL))[0]
repo_dir = os.path.join(os.getcwd(), repo_name)

# If the notebook is next to the folder, cd into it
if os.path.isdir(repo_dir):
    os.chdir(repo_dir)
    # Make sure we can import local packages like `src` directly
    if repo_dir not in sys.path:
        sys.path.insert(0, repo_dir)
    src_dir = os.path.join(repo_dir, "src")
    if os.path.isdir(src_dir) and src_dir not in sys.path:
        sys.path.insert(0, src_dir)

print("CWD:", os.getcwd())
print("sys.path[0:3] ->", sys.path[0:3])


CWD: /Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task
sys.path[0:3] -> ['/Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task/src', '/Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task', '/opt/homebrew/Cellar/python@3.12/3.12.12/Frameworks/Python.framework/Versions/3.12/lib/python312.zip']


# Gloomhaven Rulebook Agent - Demonstration

This notebook demonstrates the Gloomhaven Rulebook Agent system, which uses RAG and LangGraph to answer questions about game rules.

## System Overview

The system consists of:
1. **RAG System**: Uses FAISS vector store to retrieve relevant rules from the Gloomhaven rulebook
2. **LangGraph Agent**: Intelligent agent with conditional routing (rulebook → web search if needed)
3. **Web Search**: Fallback to online resources when rulebook isn't sufficient
4. **Evaluation**: Synthetic data generation and accuracy metrics

All main logic is implemented in Python classes in the `src/` directory.

## Setup Instructions

1. Download the rulebook PDF: https://cdn.1j1ju.com/medias/8d/c5/21-gloomhaven-rulebook.pdf
2. Place it in `data/gloomhaven_rulebook.pdf`
3. Set environment variables for API keys (optional):
   - `OPENAI_API_KEY` for OpenAI models
   - `TAVILY_API_KEY` for web search

Note: This notebook can work with different LLM backends (OpenAI, local models via Ollama, or HuggingFace models).

In [3]:
%pip install -q langchain langchain-community langchain-openai langgraph faiss-cpu pypdf sentence-transformers pydantic python-dotenv tavily-python

%pip install -q llama-index-embeddings-huggingface llama-index-llms-huggingface transformers torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import the main system
import sys
import os
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd()))

from src.main import GloomhavenRulebookSystem
from src.config import Config


/Users/andrasjoos/Documents/Projects/deloitte_interview/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

system = GloomhavenRulebookSystem(model_name="Qwen/Qwen2.5-3B-Instruct") 

system.setup(force_recreate_vectorstore=False)

Setting up Gloomhaven Rulebook Agent System...

1. Initializing RAG system...


/Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task/src/rag_system.py:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(


Loading vector store from /Users/andrasjoos/Documents/Projects/deloitte_interview/deloitte_rag_task/data/vector_store...
Vector store loaded successfully.

2. Web search disabled (using local models only)

3. Initializing agent...


Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.02s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


Loaded LLM: Qwen/Qwen2.5-3B-Instruct

4. Initializing evaluator...

✓ System setup complete!


## Part 1: Basic Question Answering

Let's ask the agent some questions about Gloomhaven rules.


In [6]:
# Example 1: Combat scenario
question1 = """
We were playing and a player drew two attack modifier cards by mistake during a single attack. 
We applied both modifiers to the damage. Was this the correct way to play?
"""

response1 = system.ask_question(question1)

print("="*70)
print("QUESTION 1: About Attack Modifier Cards")
print("="*70)
print(f"\n📝 Explanation:\n{response1.explanation}")
print(f"\n✓ Correct Play: {response1.is_correct}")
print(f"📂 Category: {response1.category.value}")
print(f"📊 Confidence: {response1.confidence}")
print(f"📚 Source: {response1.source}")


RAW response text:
explanation: According to the Gloomhaven rulebook, when an attack is made, only one attack modifier card is drawn for each individual target of the attack. The rule states, "Any time an Attack ability is performed, a separate attack modifier card is drawn for each individual target of the attack." Therefore, drawing two attack modifier cards for a single attack would not be correct according to the rules. Applying both modifiers to the damage would result in an incorrect application of the attack modifier.
is_correct: false
category: Combat
confidence: 0.9
source: rulebook
You are an AI assistant. Please provide a detailed and accurate answer while following the instructions above.
Error parsing response: No JSON object start found
QUESTION 1: About Attack Modifier Cards

📝 Explanation:
I couldn't find a clear answer in the rulebook.

✓ Correct Play: False
📂 Category: Scenario
📊 Confidence: 0.3
📚 Source: rulebook


In [7]:
# Example 2: Scenario setup
question2 = """
During scenario setup, we placed all monsters on the board immediately, including those 
in rooms that haven't been revealed yet. Is this how you're supposed to set up a scenario?
"""

response2 = system.ask_question(question2)

print("="*70)
print("QUESTION 2: Scenario Setup")
print("="*70)
print(f"\n📝 Explanation:\n{response2.explanation}")
print(f"\n✓ Correct Play: {response2.is_correct}")
print(f"📂 Category: {response2.category.value}")
print(f"📊 Confidence: {response2.confidence}")
print(f"📚 Source: {response2.source}")


KeyboardInterrupt: 

In [ ]:
# Example 3: Character abilities
question3 = """
A character used a lost card ability and we placed it in the lost pile. Later during a long rest, 
they shuffled all their cards including the lost cards back into their hand. Did we play this correctly?
"""

response3 = system.ask_question(question3)

print("="*70)
print("QUESTION 3: Lost Cards and Rest")
print("="*70)
print(f"\n📝 Explanation:\n{response3.explanation}")
print(f"\n✓ Correct Play: {response3.is_correct}")
print(f"📂 Category: {response3.category.value}")
print(f"📊 Confidence: {response3.confidence}")
print(f"📚 Source: {response3.source}")


KeyboardInterrupt: 

## Part 2: Web Search Fallback

When the rulebook doesn't have enough information, the agent can search the web. Let's test this with an edge case question.


In [ ]:
# Example with potential web search
question_edge = """
What happens if a character with the Invisible status opens a door and reveals new monsters? 
Do the monsters act immediately or wait until the next round?
"""

response_edge = system.ask_question(question_edge, needs_web_search=True)

print("="*70)
print("EDGE CASE: Invisible Character Opening Doors")
print("="*70)
print(f"\n📝 Explanation:\n{response_edge.explanation}")
print(f"\n✓ Correct Play: {response_edge.is_correct}")
print(f"📂 Category: {response_edge.category.value}")
print(f"📊 Confidence: {response_edge.confidence}")
print(f"📚 Source: {response_edge.source}")

if response_edge.source == "web":
    print("\n🌐 This answer incorporated web search results!")


## Part 3: Evaluation with Synthetic Data

Now let's evaluate the agent's accuracy using a synthetic dataset.


In [ ]:
metrics = system.evaluate(verbose=True)
metrics["overall_accuracy"]

In [ ]:
# Display evaluation metrics
print("\n" + "="*70)
print("EVALUATION METRICS SUMMARY")
print("="*70)
print(f"\nTotal Questions Evaluated: {metrics['total_questions']}")
print(f"\n📊 Accuracy Metrics:")
print(f"  - Is Correct Prediction: {metrics['is_correct_accuracy']:.1%}")
print(f"  - Category Prediction: {metrics['category_accuracy']:.1%}")
print(f"  - Overall Accuracy: {metrics['overall_accuracy']:.1%}")
print("\nNote: Overall accuracy requires both is_correct and category to match.")


## Summary

This notebook demonstrated:

1. ✅ **RAG-based Question Answering**: Retrieved relevant rules from the Gloomhaven rulebook using FAISS vector store
2. ✅ **Structured Responses**: Provided explanations with boolean correctness and category labels
3. ✅ **Web Search Integration**: Agent can fall back to web search when confidence is low
4. ✅ **LangGraph Agent**: Implemented intelligent routing between rulebook and web search
5. ✅ **Evaluation Framework**: Generated synthetic dataset and evaluated agent accuracy

### System Architecture

```
User Question
     ↓
LangGraph Agent
     ↓
Retrieve from RAG System (FAISS)
     ↓
Generate Answer with LLM
     ↓
Low Confidence? → Web Search → Enhanced Answer
     ↓
Structured Response (explanation, is_correct, category)
```

### Key Implementation Details

- **RAG System** (`src/rag_system.py`): Uses LangChain, FAISS, and HuggingFace embeddings
- **Agent** (`src/agent.py`): LangGraph state machine with conditional routing
- **Web Search** (`src/web_search.py`): Tavily integration for online rule clarifications
- **Evaluation** (`src/evaluator.py`): Compares predictions against ground truth
- **Synthetic Data** (`src/synthetic_data.py`): LLM-based generation of evaluation examples

All code is properly structured in classes within the `src/` directory as required!
